In [14]:
import requests
import ast
import matplotlib.pyplot as plt
import numpy as np

In [8]:
input_appid="30da3a731a35e7e69285b1a5dd3244af"
def get_temperatue_by_lat_lon(input_lat,input_lon):    
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={input_lat}&lon={input_lon}&appid={input_appid}"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    binary_string=response._content

    # Decode the binary string to a regular string
    decoded_string = binary_string.decode('utf-8')

    # Convert the string to a dictionary
    dictionary = ast.literal_eval(decoded_string)
    
    data=dictionary.get("main")
    # print(dictionary.get("main"))
    temperature=(data.get("temp"),data.get("temp_min"), data.get("temp_max"))
    return temperature


In [11]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(lat,lon,recipient_email):
    try:
        sender_email = "dm6192419@gmail.com"
        sender_password = "edct mxas pkco zgbn"  # Use an app password if 2FA is enabled
       
        subject = "Weather Report by Divyansh"
        
        data=get_temperatue_by_lat_lon(lat, lon)
        temperature=data[0]
        min_temp=data[1]
        max_temp=data[2]
        
        
        body = f'''As per the given longitude and latitude, the maximum temperature is {max_temp} ,
        minimum temperatiure is {min_temp} and the current temperature is {temperature}.
        '''
        
        # Set up the MIME
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = recipient_email
        message['Subject'] = subject

        # Attach the body to the email
        message.attach(MIMEText(body, 'plain'))

        # Connect to the server and send the email
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Upgrade connection to secure encrypted SSL/TLS
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, message.as_string())
        server.quit()

        print(f"Email sent successfully to email ID : {recipient_email}")

    except Exception as e:
        print(f"Failed to send email: {e}")

        

In [12]:
print(send_email(12.9716, 77.5946, "divyansh.ku@gmail.com"))

Email sent successfully to email ID : divyansh.ku@gmail.com
None


In [16]:
karnataka_cities = {
    "Bengaluru": (12.9716, 77.5946),
    "Mysuru": (12.2958, 76.6394),
    "Mangaluru": (12.9141, 74.8560),
    "Hubballi": (15.3647, 75.1239),
    "Dharwad": (15.4589, 75.0078),
    "Belagavi": (15.8497, 74.4977),
    "Ballari": (15.1394, 76.9214),
    "Kalaburagi": (17.3297, 76.8343),
    "Davangere": (14.4640, 75.9238),
    "Shivamogga": (13.9299, 75.5681),
    "Tumakuru": (13.3392, 77.1130),
    "Udupi": (13.3409, 74.7421),
    "Bidar": (17.9139, 77.5045),
    "Chikkamagaluru": (13.3166, 75.7728),
    "Raichur": (16.2076, 77.3462),
    "Hassan": (13.0072, 76.0960),
    "Bijapur": (16.8302, 75.7100),
    "Kolar": (13.1367, 78.1291),
    "Mandya": (12.5211, 76.8953),
    "Bagalkot": (16.1729, 75.6615),
}

In [17]:
def get_temp_by_city(city):
    lat,lon=karnataka_cities.get(city)
    return get_temperatue_by_lat_lon(lat,lon)

In [18]:
city_data=[]
city_names=[]

for city, lat_lon in karnataka_cities.items():
    city_names.append(city)
    city_data.append(get_temp_by_city(city))

city_names=city_names[:10]
city_data=city_data[:10]

In [25]:
def create_chart(city_data, city_names):
    # Create the x positions for the cities with spacing
    x_positions = np.arange(len(city_names)) * 4  # Multiply to create spaces between cities
    
    # Separate data for temperatures
    temperatures = [data[0] for data in city_data]
    min_temperatures = [data[1] for data in city_data]
    max_temperatures = [data[2] for data in city_data]

    # Width of the bars
    bar_width = 0.8

    # Plotting each temperature type
    plt.bar(x_positions - bar_width, min_temperatures, width=bar_width, label='Min Temperature', color='blue')
    plt.bar(x_positions, temperatures, width=bar_width, label='Temperature', color='orange')
    plt.bar(x_positions + bar_width, max_temperatures, width=bar_width, label='Max Temperature', color='red')

    # Add labels, title, and legend
    plt.xlabel("Cities")
    plt.ylabel("Temperature (°C)")
    plt.title("Temperature Data of Cities")
    plt.xticks(x_positions, city_names, rotation=45)
    plt.legend()
    
    # Adjust layout to prevent clipping
    plt.tight_layout()

    
    # Save chart to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close()
    return buf


In [36]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import matplotlib.pyplot as plt
import io

def send_email_with_chart(lat, lon, recipient_email):
    try:
        sender_email = "dm6192419@gmail.com"
        sender_password = "edct mxas pkco zgbn"  # Use an app password if 2FA is enabled
       
        subject = "Weather Report by Divyansh"
        
        data=get_temperatue_by_lat_lon(lat, lon)
        temperature=data[0]
        min_temp=data[1]
        max_temp=data[2]
        
        
        raw_message = f'''As per the given longitude and latitude, the maximum temperature is {max_temp} , minimum temperatiure is {min_temp} and the current temperature is {temperature}.'''
        
        
        # Generate the chart
        chart_buf = create_chart(city_data,city_names)
        
        # Create the email
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = recipient_email
        message['Subject'] = subject

        # HTML content
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Matplotlib Chart</title>
        </head>
        <body>
            <h2>{raw_message}</h2>
            <h3>Here is the chart for temperature:</h3>
            <img src="cid:chart_image" alt="Matplotlib Chart" style="max-width: 100%; height: 100%;">
        </body>
        </html>
        """
        message.attach(MIMEText(html_content, 'html'))

        # Attach the chart as an image
        image = MIMEImage(chart_buf.read(), name="chart.png")
        image.add_header('Content-ID', '<chart_image>')  # Match 'cid' in the HTML
        message.attach(image)

        # Connect to the server and send the email
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(message)
        server.quit()
        
        chart_buf.close()
        return "Email with chart sent successfully!"
        
        

    except Exception as e:
        return f"Failed to send email: {e}"

In [38]:
send_email_with_chart(12.9716, 77.5946, "apoorvmathur@live.in")

'Email with chart sent successfully!'